In [5]:
run functions_matching.ipynb

/eos/user/i/icortino/lhcbStyle.C:7:9: error: redefinition of 'black'
  Int_t black  = 1;
        ^
/eos/user/i/icortino/lhcbStyle.C:7:9: note: previous definition is here
  Int_t black  = 1;
        ^


In [2]:
run configurations_matching.ipynb

In [8]:
smearing = False
percentage = 97

##### INITIALISE VARIABLES #####

timestart = time.time()

nVeloUTTracks = 0
nentries = ch1.GetEntries()

index_perc = perc_list.index(percentage)
#threshold_chi2 = thr_nosm_list[index_perc]
threshold_chi2 = 6000

#lists for slopes
txVeloVelo_list=[]
tyVeloVelo_list=[]
txVeloUT_list=[]
tyVeloUT_list=[]
txUTUT_list=[]
tyUTUT_list=[]

difftx_vv_uu_list = []
diffty_vv_uu_list = []

p_list=[]
p_list_10 = []

print '\n\n\n\npercentage: ', percentage 
print 'smearing: ', smearing 
print 'threshold chi2: : ', threshold_chi2 

##### LOOP #####

for event in ch1:
    
    #positive charge
    if event.qop < 0 and event.p>5000:
        
        velolistsxyz = removezeros(event.HitVeloXpos,event.HitVeloYpos,event.HitVeloZpos)
        velolistsx = velolistsxyz[0];velolistsy = velolistsxyz[1];velolistsz = velolistsxyz[2]

        utlistsxyz = removezeros(event.HitUTXpos,event.HitUTYpos,event.HitUTZpos)
        utlistsx = utlistsxyz[0];utlistsy = utlistsxyz[1];utlistsz = utlistsxyz[2]
        
        if (isvelovelo(utlistsz,velolistsz)) == 'VeloVelo' and (isutut(utlistsz,velolistsz)) == 'UTUT':
            nVeloUTTracks +=1
            

            #save fist two hits in velo and first two in UT
            xu0=utlistsx[0];  yu0 = utlistsy[0]; zu0 = utlistsz[0];
            xu1=utlistsx[len(utlistsz)-1];  yu1 = utlistsy[len(utlistsz)-1]; zu1 = utlistsz[len(utlistsz)-1];
            
            xv0=velolistsx[0];  yv0 = velolistsy[0]; zv0 = velolistsz[0];
            xv1=velolistsx[len(velolistsz)-1];  yv1 = velolistsy[len(velolistsz)-1]; zv1 = velolistsz[len(velolistsz)-1];
            
            slopevv = Slopexy(xv0,yv0,zv0,xv1,yv1,zv1)
            slopevu = Slopexy(xv0,yv0,zv0,xu0,yu0,zu0)
            slopeuu = Slopexy(xu0,yu0,zu0,xu1,yu1,zu1)

            txVeloVelo_list.append(slopevv[0])
            tyVeloVelo_list.append(slopevv[1])
            txVeloUT_list.append(slopevu[0])
            tyVeloUT_list.append(slopevu[1])
            txUTUT_list.append(slopeuu[0])
            tyUTUT_list.append(slopeuu[1])

            difftx_vv_uu_list.append(slopevv[0]-slopeuu[0])
            diffty_vv_uu_list.append(slopevv[1]-slopeuu[1])

            p_list.append(event.p)
        
    if nVeloUTTracks > 10000:
        break
        
    
        
        







percentage:  97
smearing:  False
threshold chi2: :  6000


In [9]:
Nbins=1000

print(len(p_list))
print(len(p_list_10))

difftx_vvuu=TH2F("difftx_vvuu", "", Nbins,5000,50000,Nbins,-0.05,0.05)
diffty_vvuu=TH2F("diffty_vvuu", "", Nbins,5000,50000,Nbins,-0.05,0.05)

reldifftx_vvuu=TH2F("reldifftx_vvuu", "", Nbins,5000,50000,Nbins,-1,1)
reldiffty_vvuu=TH2F("reldiffty_vvuu", "", Nbins,5000,50000,Nbins,-1,1)


#th2f p vs diff_vvuu
for index in range(0,len(p_list)):
    difftx_vvuu.Fill(p_list[index],difftx_vv_uu_list[index])
    diffty_vvuu.Fill(p_list[index],diffty_vv_uu_list[index])
    if(txVeloVelo_list[index] != 0):
        reldifftx_vvuu.Fill(p_list[index],difftx_vv_uu_list[index]/txVeloVelo_list[index])
    if(tyVeloVelo_list[index] != 0):
        reldiffty_vvuu.Fill(p_list[index],diffty_vv_uu_list[index]/tyVeloVelo_list[index])
    

    
Nbins = 60
tx_vv=TH1F("tx_vv", "", Nbins,-0.25,0.25)
ty_vv=TH1F("ty_vv", "", Nbins,-0.25,0.25)

#th1f slope vv
for index in range(0,len(p_list)):
    tx_vv.Fill(txVeloVelo_list[index])
    ty_vv.Fill(tyVeloVelo_list[index])
    


3730
0


Warning in <TROOT::Append>: Replacing existing TH1: difftx_vvuu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: diffty_vvuu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: reldifftx_vvuu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: reldiffty_vvuu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: tx_vv (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ty_vv (Potential memory leak).


In [10]:
c4=TCanvas("c4","",1500,600)
c4.Divide(2,1)

c4.cd(1)
difftx_vvuu.Draw()
difftx_vvuu.GetXaxis().SetTitle("Momentum [MeV]")
difftx_vvuu.GetYaxis().SetTitle("Difference in xz slope")
difftx_vvuu.SetMarkerSize(0.5)

c4.cd(2)
diffty_vvuu.Draw()
diffty_vvuu.GetXaxis().SetTitle("Momentum [MeV]")
diffty_vvuu.GetYaxis().SetTitle("Difference in yz slope")
diffty_vvuu.SetMarkerSize(0.5)

'''c4.cd(3)
reldifftx_vvuu.Draw()
reldifftx_vvuu.GetXaxis().SetTitle("p")
reldifftx_vvuu.GetYaxis().SetTitle("difftx_vvuu/txvv")
reldifftx_vvuu.SetMarkerSize(0.5)

c4.cd(4)
reldiffty_vvuu.Draw()
reldiffty_vvuu.GetXaxis().SetTitle("p")
reldiffty_vvuu.GetYaxis().SetTitle("diffty_vvuu/tyvv")
reldiffty_vvuu.SetMarkerSize(0.5)


c4.cd(3)
tx_vv.Draw()
tx_vv.GetXaxis().SetTitle("tx_vv")
tx_vv.GetYaxis().SetTitle("Events")

c4.cd(4)
ty_vv.Draw()
ty_vv.GetXaxis().SetTitle("tx_vv")
ty_vv.GetYaxis().SetTitle("Events")'''


c4.SaveAs("slopes_vvuu.png")




Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Info in <TCanvas::Print>: png file slopes_vvuu.png has been created
